In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def fetchGHCNStationData(stationId):
    df = pd.read_csv(
        f"http://noaa-ghcn-pds.s3.amazonaws.com/csv/by_station/{stationId}.csv"
    )

    df = df[["DATE", "ELEMENT", "DATA_VALUE"]]
    df = df[(df["DATE"] >= 19910101) & (df["DATE"] <= 20201231)]
    df = df[df["ELEMENT"].isin(["TMAX", "TMIN"])]
    df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")

    pivot = df.pivot(index="DATE", columns="ELEMENT", values="DATA_VALUE").reset_index()
    pivot["TMAX"] = pivot["TMAX"] / 10
    pivot["TMIN"] = pivot["TMIN"] / 10
    pivot["md"] = pivot["DATE"].dt.strftime("%m-%d")

    dailyagg = pivot.groupby(["md"], as_index=False).agg(
        record_max_temp=("TMAX", "max"),
        record_min_temp=("TMIN", "min"),
        average_max_temp=("TMAX", "mean"),
        average_min_temp=("TMIN", "mean"),
    )

    return dailyagg

In [3]:
avl_stid = "USW00003812"
df = fetchGHCNStationData(avl_stid)
df

/var/folders/8t/4dcpfx6j29d858grh_vp_d_m0000gn/T/ipykernel_75689/2318973544.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,md,record_max_temp,record_min_temp,average_max_temp,average_min_temp
0,01-01,19.4,-11.7,10.083333,-1.400000
1,01-02,19.4,-13.2,9.016667,-0.806667
2,01-03,22.8,-12.2,8.410000,-1.306667
3,01-04,21.7,-11.1,9.426667,-1.670000
4,01-05,19.4,-12.2,9.073333,-1.653333
...,...,...,...,...,...
361,12-27,21.7,-8.9,10.300000,-1.433333
362,12-28,19.4,-9.4,10.130000,-0.766667
363,12-29,21.7,-8.3,11.466667,0.190000
364,12-30,20.0,-8.9,9.620000,-1.226667
